# Урок 3. Классификация. Логистическая регрессия

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
X = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
              [1, 1, 2, 5, 3, 0, 5, 10, 1, 2],
              [500, 700, 750, 600, 1450, 800, 1500, 2000, 450, 1000],
              [1, 1, 2, 1, 2,  1, 3, 3, 1, 2]], dtype = np.float64)
y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1], dtype = np.float64)

In [3]:
def calc_std_feat(x):
    res = (x - x.mean()) / x.std()
    return res

In [4]:
X_st = X.copy()
X_st[2, :] = calc_std_feat(X[2, :])

In [5]:
def sigmoid(z):
    res = 1 / (1 + np.exp(-z))
    return res

## Практическое задание

### Задание 1

Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log.

### Ответ 1

In [11]:
def calc_logloss(y, y_pred):
    y_pred = np.clip(y_pred, a_min=1e-10, a_max=1-1e-10)
    err = np.mean(-y * np.log(y_pred) - (1.0-y) * np.log(1.0-y_pred))
    err = np.sum(err)
    return err

# Пример применения
y1 = np.array([1, 0])
y_pred1 = np.array([0.9, 0])
calc_logloss(y1, y_pred1)

0.052680257878913145

### Задание 2

Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.

### Ответ 2

In [16]:
def eval_model(X, y, iterations, alpha=1e-4):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    for i in range(1, iterations+1):
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        W -= alpha * (1/n * np.dot((y_pred - y), X.T))
        if i % (iterations / 10) == 0:
            print(i, W, err)
            
    return W, err

# параметры, при которых минимум потерь
min_iterations = None 
min_alpha = None
min_err = None
min_W = None

for i in range(80000, 120000, 1000):
    alpha = i/10000 
    for iterations in range(1, 100000, 10000):
        W, err = eval_model(X_st, y, iterations=iterations, alpha=alpha)
        if min_err is None or err < min_err:
            min_err = err
            min_alpha = alpha
            min_iterations = iterations
            min_W = W
            
print(f'err = {min_err}')
print(f'alpha = {min_alpha}')
print(f'iterations = {min_iterations}')
print(f'W = {min_W}')

err = 0.00618465627491236
alpha = 11.9
iterations = 90001
W = [-228.51937215  -16.07109826  -67.74535072  174.76775113]


### Задание 3

Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1. На вход подаётся W, который уже посчитан функцией eval_model, и X, на выходе — массив y_pred_proba.

### Ответ 3

In [23]:
def calc_pred_proba(w, X):
    return sigmoid(np.dot(w, X))

y_pred_proba = calc_pred_proba(min_W, X_st)
y_pred_proba

array([3.04698996e-02, 2.30239962e-14, 1.00000000e+00, 3.24629545e-36,
       9.98406437e-01, 1.88020623e-13, 1.00000000e+00, 2.94273034e-04,
       9.71403833e-01, 1.00000000e+00])

### Задание 4

Создайте функцию calc_pred, возвращающую предсказанный класс. На вход подаётся W, который уже посчитан функцией eval_model, и X, на выходе — массив y_pred.

### Ответ 4

In [25]:
def calc_pred(w, X):
    pred = sigmoid(np.dot(w, X))
    pred.round(2)
    return np.where(pred > 0.5, 1, 0)

y_pred = calc_pred(min_W, X_st)
y_pred

array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1])

### Задание 5

Посчитайте Accuracy, матрицу ошибок, точность и полноту, а также F1 score.

### Ответ 5

In [26]:
from sklearn.metrics import accuracy_score
accuracy_score(y, y_pred)

1.0

In [27]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y, y_pred)

array([[5, 0],
       [0, 5]])

In [29]:
from sklearn.metrics import f1_score
f1_score(y, y_pred)

1.0

### Задание 6

Могла ли модель переобучиться? Почему?

### Ответ 6

Слишком тщательный подход к поиску значений гиперпараметров привёл к переобучению на тестовой выборке. Теперь модель будет плохо работать на будущих данных.